In [1]:
    try:
        from .markdown import renderer, CommonMark
        from .environment import environment
    except:
        from markdown import renderer, CommonMark
        from environment import environment

        import IPython
    import types, sys, IPython, asyncio, trio, traceback, ipykernel, traitlets, nbconvert, jinja2, operator, toolz.curried as toolz, tokenize, io, textwrap
    ip = IPython.get_ipython()


    import toolz.curried as toolz

In [2]:
    from dataclasses import dataclass, field

In [3]:
    tangle = renderer

In [4]:
    try:
        from .user_expressions import user_expressions, evaluate_expression, display_expression_results
    except:
        from user_expressions import user_expressions, evaluate_expression, display_expression_results

In [6]:
    import jinja2

In [23]:
    @dataclass
    class Weave:
        """A contextmanager to initialize and finalize a weave step.  
        """
        source: str = ""
        block: str = ""
        inline: tuple = field(default_factory=tuple)
        template: bool = True
        markdown: bool = True
        namespace: dict = field(default_factory=dict, repr=False)
        display: bool = True
        environment: jinja2.Environment = field(default=environment)
        ip: IPython.core.interactiveshell.InteractiveShell = field(default_factory=get_ipython)        
        
        def __post_init__(self):
            if self.markdown:
                self.block, inline = renderer(self.source)
                if self.inline is not False:
                    self.inline = inline
            else:
                self.block = self.block or self.source
                if self.inline is True:
                    self.inline = set()                
            self.namespace = self.namespace or self.ip.user_ns
            
            if self.display is True:
                self.display = IPython.core.display.DisplayHandle()
        
        def __enter__(self):
            self.display and IPython.display.display(self)
            return self
        
        def __exit__(self, *exc):
            if all(e is None for e in exc): trio.run(self.expressions)
                    
        async def expressions(self, *futures): 
            async with trio.open_nursery() as nursery:
                if self.inline:
                    for expression in set(self.inline):
                        nursery.start_soon(
                            evaluate_expression, self.ip, expression, 
                            self.namespace, self.namespace)
                
                self.template and nursery.start_soon(self.render)
                    
        async def render(self):
            try:
                self.display and self.display.update(
                    IPython.display.Markdown(
                        self.environment.from_string(self.source).render(**self.namespace)
                    ))
            except: __import__('traceback').print_exc()
                    
        def _ipython_display_(self):
            if not self.source.splitlines()[0].strip(): return
            if self.source.lstrip().startswith('%%'): return
            if textwrap.dedent(self.source).strip() == self.block.strip(): return
            if self.display is False: return
            self.display.display(IPython.display.Markdown(self.source))